In [ ]:
! pip install transformers datasets torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import Trainer, TrainingArguments,AutoTokenizer, AutoModelForCausalLM

# Load CSV
csv_file = 'grades.csv'  # Replace with your CSV file path
df = pd.read_csv(csv_file)

# Create a function to generate Q&A pairs
def create_qa_pairs(row):
    qa_pairs = []
    qa_pairs.append({
        "input": f"What is the first name of {row['Last name']}?",
        "output": row['First name']
    })
    qa_pairs.append({
        "input": f"What is the SSN of {row['Last name']} {row['First name']}?",
        "output": row['SSN']
    })
    qa_pairs.append({
        "input": f"What was the Test1 score of {row['Last name']}?",
        "output": str(row['Test1'])
    })
    qa_pairs.append({
        "input": f"What was the final grade of {row['Last name']}?",
        "output": row['Grade']
    })
    # Add more pairs for other fields if necessary
    return qa_pairs

qa_data = []
for _, row in df.iterrows():
    qa_data.extend(create_qa_pairs(row))

# Convert to a Dataset
dataset = Dataset.from_pandas(pd.DataFrame(qa_data))

def preprocess_function(examples):
    inputs = []
    outputs = []
    # Iterate over the items in the 'examples' dictionary
    for example in examples['input']:
        inputs.append(example)
    for example in examples['output']:
        outputs.append(example)

    model_inputs = tokenizer(inputs, truncation=True, padding='max_length', return_tensors='pt')
    labels = tokenizer(outputs, truncation=True, padding='max_length', return_tensors='pt')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")
tokenizer.pad_token = tokenizer.eos_token

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

Dataset({
    features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 64
})

In [ ]:
from transformers import DataCollatorForLanguageModeling
model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
training_args = TrainingArguments(
    output_dir="./result",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    label_smoothing_factor=0.0 # Remove label smoothing
)
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator
)

# Train the model
trainer.train()

# Save the model
#model.save_pretrained("./fine_tuned_distill_gpt_model")

Step,Training Loss


TrainOutput(global_step=96, training_loss=2.9896227518717446, metrics={'train_runtime': 49.6987, 'train_samples_per_second': 3.863, 'train_steps_per_second': 1.932, 'total_flos': 50168976113664.0, 'train_loss': 2.9896227518717446, 'epoch': 3.0})

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=96, training_loss=0.8750318686167399, metrics={'train_runtime': 43.6335, 'train_samples_per_second': 4.4, 'train_steps_per_second': 2.2, 'total_flos': 50168976113664.0, 'train_loss': 0.8750318686167399, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./distill_gpt2")

In [ ]:
# !pip install huggingface_hub

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

In [ ]:
model.push_to_hub("ahmadmac/DistillGPT2-CSV")
tokenizer.push_to_hub("ahmadmac/DistillGPT2-CSV")

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ahmadmac/DistillGPT2/commit/31e2afefc13c3e9920784241e769b4542916d194', commit_message='Upload tokenizer', commit_description='', oid='31e2afefc13c3e9920784241e769b4542916d194', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("ahmadmac/DistillGPT2-CSV")
model = AutoModelForCausalLM.from_pretrained("ahmadmac/DistillGPT2-CSV")

tokenizer_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
def get_response(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output = model.generate(input_ids, max_length=32, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response
print(get_response("What is the First name of Buff?"))


What is the First name of Buff?


























In [ ]:
gemini_model = genai.GenerativeModel('gemini-1.5-pro')

def get_response(input_text):

    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    trained_output = model.generate(input_ids, max_length=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    trained_response = tokenizer.decode(trained_output[0], skip_special_tokens=True)

    prompt = f"Improve this text to make it clearer and more concise: {trained_response}"
    genai_response = gemini_model.generate_content(prompt)

    return genai_response
prompt_text = "What is the First name of Buff?"
improved_response = get_response(prompt_text)
print(improved_response.text)

This question needs more context.  "Buff" on its own isn't a common name or term that implies someone's first name. 

To help me understand what you mean, please tell me:

* **Who or what is "Buff" referring to?**  Is it a person, a character, an animal, a product, etc.?
* **Are there any other details you can provide?**  For example, a last name, a profession, or the context where you encountered the name "Buff." 

Once I have more information, I can help you figure out the first name. 😊 

